Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [1]:
from collections import Counter
import numpy as np

In [2]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [3]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player): #check if a player has won
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

## Montecarlo Evaluation

In [4]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, player):
        # Alice won
        return 1
    elif four_in_a_row(board, -player):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)
        #return 0

## Example

In [5]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1)
play(board, 0, -1)
play(board, 4, 1)
play(board, 0, -1)
play(board, 5, 1)
print(board)
eval_board(board, 1)


[[-1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


0.86

# MinMax 
### Using Alpha-Beta pruning and  Monte Carlo simulation

In [6]:
def display(board):
    """ Display the board using a standard view representation and using X and O as disks of the two players"""
    print()
    b = list(np.flipud(board.T).flatten())
    buffer = "|"
    for i in range(len(b)):
        buffer += "X" if b[i] == 1 else "O" if b[i] == -1 else " "
        if (i + 1) % 7 == 0 and i != len(b) - 1:
            buffer += "|\n|"
    buffer += "|"
    print(buffer)
    print()

def flip_board(b):
    """ Change all the X disks in O disks and vice versa"""
    board = b.copy()
    board[b == -1] = 1
    board[b == 1] = -1
    return board


def minmax(board, depth, max_depth, player, alpha, beta, maximization=True):
    """ MinMax algortihm using an hard cut off, Alpha-Beta pruning and a heuristic evaluation (MonteCarlo)"""
    val = eval_board(board,player)
    possible = valid_moves(board)
    if depth != max_depth and (val ==1 or val==-1 or not possible or depth == 0) :
        #if depth == 0 and possible:
            #print("Hard cut off")
        return None, val
    evaluations = list()
    b = flip_board(board)
    for ply in possible:
        play(b, ply, -player)
        _, val = minmax(b, depth-1, max_depth, player, alpha, beta, not maximization)
        take_back(b,ply)
        evaluations.append((ply, -val))
        # Alfa-Beta pruning
        if maximization:
            alpha = max(alpha, -val)
        else:
            beta = max(beta, -val)
        if alpha >= -beta:
            break
    return max(evaluations, key=lambda k: k[1])

In [10]:
#board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)

# Board is here defined with a "normal view"
board = [[ 0, 0, 0, 0, 0, 0, 0],
		 [ 0, 1,-1, 1, 1, 1, 0],
		 [ 1, 1, 1,-1,-1, 1, 1],
		 [-1,-1, 1, 1,-1,-1,-1],
		 [ 1, 1,-1,-1,-1, 1,-1],
		 [-1,-1,-1, 1, 1,-1, 1]]

# We adapt the board for the internal representation
board = np.array(board)
board = np.fliplr(board.T)

display(board)
if four_in_a_row(board, 1):
    print("Player 1 wins!!")
elif four_in_a_row(board, -1):
    print("Player 2 wins!!")

PLAYER = 1

print("X turn" if PLAYER==1 else "O turn")

best_ply, eval = minmax(board, 3, 3, PLAYER, -np.inf, -np.inf, True)

print(f"Best ply: {best_ply}, eval: {eval}")

play(board,best_ply,PLAYER)

display(board)
print()
if four_in_a_row(board, 1):
    print("Player 1 wins!!")
elif four_in_a_row(board, -1):
    print("Player 2 wins!!")


|       |
| XOXXX |
|XXXOOXX|
|OOXXOOO|
|XXOOOXO|
|OOOXXOX|

X turn
Best ply: 6, eval: 1

|       |
| XOXXXX|
|XXXOOXX|
|OOXXOOO|
|XXOOOXO|
|OOOXXOX|


Player 1 wins!!


# Monte Carlo Tree Search

In [13]:
import numpy as np

dx = [ 1, 1,  1,  0 ]
dy = [ 1, 0,  -1,  1  ]

class Board:

    def __init__(self, board , last_move = [ None , None ] ):
        self.board = board 
        self.last_move = last_move
    
    def __str__(self):
        b = list(np.flipud(self.board.T).flatten())
        buffer = "|"
        for i in range(len(b)):
            buffer += "X" if b[i] == 1 else "O" if b[i] == -1 else " "
            if (i + 1) % 7 == 0 and i != len(b) - 1:
                buffer += "|\n|"
        buffer += "|"
        return buffer

    def play(self, column, player):
        """Updates `board` as `player` drops a disc in `column`"""
        (index,) = next((i for i, v in np.ndenumerate(self.board[column]) if v == 0))
        self.board[column, index] = player

    def terminal(self):
        """ Returns true when the game is finished, otherwise false."""
        for i in range(len(self.board[:,5])):
       		if ( self.board[i][5] == 0 ):
       			return False
        return True

    def winner(self):
        """Takes the board as input and determines if there is a winner.
           If the game has a winner, it returns the player number (Computer = 1, Human = -1).
           If the game is still ongoing, it returns zero."""
        x = self.last_move[0]
        y = self.last_move[1]

        if x == None:
            return 0 

        for d in range(4):

            h_counter = 0
            c_counter = 0

            for k in range(-3,4):

                u = x + k * dx[d]
                v = y + k * dy[d]

                if u < 0 or u >= 6:
                    continue

                if v < 0 or v >= 7:
                    continue

                if self.board.T[u][v] == -1:
                    c_counter = 0
                    h_counter += 1
                elif self.board.T[u][v] == 1:
                    h_counter = 0
                    c_counter += 1
                else:
                    h_counter = 0
                    c_counter = 0

                if h_counter == 4:
                    return -1 

                if c_counter == 4:	
                    return 1

        return 0

    def valid_moves(self):
        """Returns columns where a disc may be played"""
        return [n for n in range(NUM_COLUMNS) if self.board[n, COLUMN_HEIGHT - 1] == 0]

    def next_state(self, player):
        """Retuns next state"""
        aux_board = self.board.copy()
        aux = Board(aux_board)
        moves = aux.valid_moves()
        if len(moves) > 0 :
            c = np.random.choice(moves)
            aux.play(c, player)
            row = len(aux.board[c,:])-sum(aux.board[c,:] == 0)-1
            aux.last_move = [ row, c ]
        return aux

## Monte Carlo Tree Search

class Node:
# Data structure to keep track of our search
    def __init__(self, state, parent = None):
        self.visits = 1 
        self.reward = 0.0
        self.state = state
        self.children = []
        self.children_move = []
        self.parent = parent 

    def addChild( self , child_state , move ):
        child = Node(child_state,self)
        self.children.append(child)
        self.children_move.append(move)

    def update( self,reward ):
        self.reward += reward 
        self.visits += 1

    def fully_explored(self):
        if len(self.children) == len(self.state.valid_moves()):
            return True
        return False

def MTCS( maxIter , root , factor, player ):
    for _ in range(maxIter):
        front, turn = treePolicy( root , player , factor )
        reward = defaultPolicy(front.state, turn)
        backup(front,reward,turn)

    ans = bestChild(root,0)
    print("\nWinrate for each column:")
    print([(c.reward/c.visits) for c in ans.parent.children ])
    return ans


def treePolicy( node, turn , factor ):
    while node.state.terminal() == False and node.state.winner() == 0:
        if ( node.fully_explored() == False ):
            return expand(node, turn), -turn
        else:
            node = bestChild ( node , factor )
            turn *= -1
    return node, turn

def expand( node, turn ):
    tried_children_move = [m for m in node.children_move]
    possible_moves = node.state.valid_moves()

    for move in possible_moves:
        if move not in tried_children_move:
            aux_board = node.state.board.copy()
            new_state = Board(aux_board)
            new_state.play(move, turn)
            row = len(new_state.board[move,:])-sum(new_state.board[move,:] == 0)-1
            new_state.last_move = [ row, move ]
            break

    node.addChild(new_state,move)
    return node.children[-1]

def bestChild(node,factor):
    bestscore = -10000000.0
    bestChildren = []
    for c in node.children:
        exploit = c.reward / c.visits
        explore = np.sqrt(np.log(2.0*node.visits)/float(c.visits))
        score = exploit + factor*explore
        if score == bestscore:
            bestChildren.append(c)
        if score > bestscore:
            bestChildren = [c]
            bestscore = score 
    return np.random.choice(bestChildren)

def defaultPolicy( state, turn  ):
    while state.terminal()==False and state.winner() == 0 :
        state = state.next_state( turn )
        turn *= -1
    return  state.winner() # return the reward of the leaf state

def backup( node , reward, turn ):
    while node != None:
        node.visits += 1 
        node.reward -= turn*reward
        node = node.parent
        turn *= -1
    return

class Game:
    def __init__(self, board):
        self.end = False
        board = np.fliplr(np.array(board).T)
        self.b = Board( board )

    def findBestMove(self, factor, player):
        """Returns the best move using MonteCarlo Tree Search"""
        o = Node(self.b)
        bestMove = MTCS( 3000, o, factor, player)
        self.b = Board(bestMove.state.board.copy())
        self.b.last_move = bestMove.state.last_move

    def display_board(self):
        """Board visual representation"""
        print()
        b = list(np.flipud(self.b.board.T).flatten())
        buffer = "|"
        for i in range(len(b)):
           buffer += "X" if b[i] == 1 else "O" if b[i] == -1 else " "
           if (i + 1) % 7 == 0 and i != len(b) - 1:
              buffer += "|\n|"
        buffer += "|"
        print(buffer)
        print()
    
    def status(self):
        """Status of the game"""
        if not self.end:
            result = self.b.winner()
            if result == 1 or result == -1:
                print(f"Player {result} wins!!")
                self.end = True
            elif self.b.terminal():
                print("Draw")
                self.end = True
        else:
            print("End game")



if __name__ == "__main__":

    board = [[ 0, 0, 0, 0, 0, 0, 0],
			 [ 0, 1,-1, 1, 1, 1, 0],
		 	 [ 1, 1, 1,-1,-1, 1, 1],
		     [-1,-1, 1, 1,-1,-1,-1],
		 	 [ 1, 1,-1,-1,-1, 1,-1],
			 [-1,-1,-1, 1, 1,-1, 1]]

    g = Game(board)
    g.display_board()

    PLAYER = 1
    print("X turn" if PLAYER==1 else "O turn")

    g.findBestMove(2.0, PLAYER)
    g.display_board()
    g.status()


|       |
| XOXXX |
|XXXOOXX|
|OOXXOOO|
|XXOOOXO|
|OOOXXOX|

X turn

Winrate for each column:
[-0.25, 0.8588235294117647, -0.2777777777777778, -0.125, -0.2777777777777778, -0.2631578947368421, 0.9995829858215179]

|       |
| XOXXXX|
|XXXOOXX|
|OOXXOOO|
|XXOOOXO|
|OOOXXOX|

Player 1 wins!!
